In [1]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict

from data_utlis.dataset import MyDataset
from utils import to_var, idx2word, expierment_name
from models.model import SentenceVAE
import losses

In [2]:
class Bunch(dict):
    def __init__(self, *args, **kwds):
        super(Bunch, self).__init__(*args, **kwds)
        self.__dict__ = self
        
def interpolate(start_val, end_val, start_iter, end_iter, current_iter):
    if current_iter < start_iter:
        return start_val
    elif current_iter >= end_iter:
        return end_val
    else:
        return start_val + (end_val - start_val) * (current_iter - start_iter) / (end_iter - start_iter)

def anneal(cfgan, it):
    return interpolate(cfgan.start.val, cfgan.end.val, cfgan.start.iter, cfgan.end.iter, it)


def main(args):
    ts = time.strftime('%Y-%b-%d-%H:%M:%S', time.gmtime())

    splits = ['train', 'valid'] + (['test'] if args.test else [])

    datasets = OrderedDict()
    for split in splits:
        datasets[split] = MyDataset(
            data_dir=args.data_dir,
            split=split,
            create_data=args.create_data,
            max_sequence_length=args.max_sequence_length,
            min_occ=args.min_occ
        )

    params = dict(
        vocab_size=datasets['train'].vocab_size,
        sos_idx=datasets['train'].sos_idx,
        eos_idx=datasets['train'].eos_idx,
        pad_idx=datasets['train'].pad_idx,
        unk_idx=datasets['train'].unk_idx,
        max_sequence_length=args.max_sequence_length,
        embedding_size=args.embedding_size,
        rnn_type=args.rnn_type,
        hidden_size=args.hidden_size,
        word_dropout=args.word_dropout,
        embedding_dropout=args.embedding_dropout,
        latent_size=args.latent_size,
        num_layers=args.num_layers,
        bidirectional=args.bidirectional
    )
    model = SentenceVAE(**params)

    if torch.cuda.is_available():
        model = model.cuda()

    print(model)

    if args.tensorboard_logging:
        writer = SummaryWriter(os.path.join(args.logdir, expierment_name(args, ts)))
        writer.add_text("model", str(model))
        writer.add_text("args", str(args))
        writer.add_text("ts", ts)

    save_model_path = os.path.join(args.save_model_path, ts)
    os.makedirs(save_model_path)

    with open(os.path.join(save_model_path, 'model_params.json'), 'w') as f:
        json.dump(params, f, indent=4)

    def kl_anneal_function(anneal_function, step, k, x0):
        if anneal_function == 'logistic':
            return float(1/(1+np.exp(-k*(step-x0))))
        elif anneal_function == 'linear':
            return min(1, step/x0)

    NLL = torch.nn.NLLLoss(ignore_index=datasets['train'].pad_idx, reduction='sum')
    def loss_fn(logp, target, length, mean, logv, anneal_function, step, k, x0):

        # cut-off unnecessary padding from target, and flatten
        target = target[:, :torch.max(length).item()].contiguous().view(-1)
        logp = logp.view(-1, logp.size(2))

        # Negative Log Likelihood
        NLL_loss = NLL(logp, target)

        # KL Divergence
        KL_loss = -0.5 * torch.sum(1 + logv - mean.pow(2) - logv.exp())
        KL_weight = kl_anneal_function(anneal_function, step, k, x0)

        return NLL_loss, KL_loss, KL_weight

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

    tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
    step = 0
    for epoch in range(args.epochs):

        for split in splits:

            data_loader = DataLoader(
                dataset=datasets[split],
                batch_size=args.batch_size,
                shuffle=split=='train',
                num_workers=cpu_count(),
                pin_memory=torch.cuda.is_available()
            )

            tracker = defaultdict(tensor)

            # Enable/Disable Dropout
            if split == 'train':
                model.train()
            else:
                model.eval()

            for iteration, batch in enumerate(data_loader):

                batch_size = batch['input'].size(0)

                for k, v in batch.items():
                    if torch.is_tensor(v):
                        batch[k] = to_var(v)

                # Forward pass
                logp, mean, logv, z = model(batch['input'], batch['length'])

                #loss calculation
                NLL_loss, KL_loss, KL_weight = loss_fn(logp, batch['target'],
                    batch['length'], mean, logv, args.anneal_function, step, args.k, args.x0)
                wae_mmd_loss = losses.wae_mmd_gaussianprior(z, method='full_kernel')
                wae_mmdrf_loss = losses.wae_mmd_gaussianprior(z, method='rf')
                z_regu_loss = wae_mmdrf_loss
                z_logvar_L1 = logv.abs().sum(1).mean(0)  # L1 in z-dim, mean over mb.
                z_logvar_KL_penalty = losses.kl_gaussian_sharedmu(mean, logv)
                lambda_logvar_L1=0.0,  # default from https://openreview.net/pdf?id=r157GIJvz
                lambda_logvar_KL=1e-3,  # default from https://openreview.net/pdf?id=r157GIJvz
                
                
                beta=Bunch(
                    start=Bunch(val=1.0, iter=0),
                    end=Bunch(val=2.0, iter=10000)
                )
                beta = anneal(beta, iteration)
                # beta=Bunch(
                #     start=Bunch(val=2.0, iter=vae.n_iter),
                #     end=Bunch(val=2.0, iter=vae.n_iter + 50000)
                # ),
                loss = NLL_loss \
                       + z_regu_loss\
                       + lambda_logvar_L1 * z_logvar_L1 \
                       + lambda_logvar_KL * z_logvar_KL_penalty
                loss = loss / batch_size
                
                # recon_loss = losses.recon_dec(inputs.text, dec_logits)
                # kl_loss = losses.kl_gaussianprior(z_mu, z_logvar)
                # wae_mmd_loss = losses.wae_mmd_gaussianprior(z, method='full_kernel')
                # wae_mmdrf_loss = losses.wae_mmd_gaussianprior(z, method='rf')
                # z_regu_losses = {'kl': kl_loss, 'mmd': wae_mmd_loss, 'mmdrf': wae_mmdrf_loss}
                # z_regu_loss = z_regu_losses[cfgv.z_regu_loss]
                # z_logvar_L1 = z_logvar.abs().sum(1).mean(0)  # L1 in z-dim, mean over mb.
                # z_logvar_KL_penalty = losses.kl_gaussian_sharedmu(z_mu, z_logvar)
                # loss = recon_loss + beta * z_regu_loss \
                #     + cfgv.lambda_logvar_L1 * z_logvar_L1 \
                #     + cfgv.lambda_logvar_KL * z_logvar_KL_penalty

                # backward + optimization
                if split == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    step += 1

                # bookkeepeing
                tracker['ELBO'] = torch.cat((tracker['ELBO'], loss.data.view(1, -1)), dim=0)

                if args.tensorboard_logging:
                    writer.add_scalar("%s/ELBO" % split.upper(), loss.item(), epoch*len(data_loader) + iteration)
                    writer.add_scalar("%s/NLL Loss" % split.upper(), NLL_loss.item() / batch_size,
                                      epoch*len(data_loader) + iteration)
                    writer.add_scalar("%s/wae_mmdrf_loss" % split.upper(), wae_mmdrf_loss.item() / batch_size,
                                      epoch*len(data_loader) + iteration)
                    # writer.add_scalar("%s/KL Weight" % split.upper(), KL_weight,
                    #                   epoch*len(data_loader) + iteration)

                if iteration % args.print_every == 0 or iteration+1 == len(data_loader):
                    print("%s Batch %04d/%i, Loss %9.4f, NLL-Loss %9.4f, wae_mmdrf_loss %9.4f"
                          % (split.upper(), iteration, len(data_loader)-1, loss.item(), NLL_loss.item()/batch_size,
                          wae_mmdrf_loss.item()/batch_size))

                if split == 'valid':
                    if 'target_sents' not in tracker:
                        tracker['target_sents'] = list()
                    tracker['target_sents'] += idx2word(batch['target'].data, i2w=datasets['train'].get_i2w(),
                                                        pad_idx=datasets['train'].pad_idx)
                    tracker['z'] = torch.cat((tracker['z'], z.data), dim=0)

            print("%s Epoch %02d/%i, Mean ELBO %9.4f" % (split.upper(), epoch, args.epochs, tracker['ELBO'].mean()))

            if args.tensorboard_logging:
                writer.add_scalar("%s-Epoch/ELBO" % split.upper(), torch.mean(tracker['ELBO']), epoch)

            # save a dump of all sentences and the encoded latent space
            if split == 'valid':
                dump = {'target_sents': tracker['target_sents'], 'z': tracker['z'].tolist()}
                if not os.path.exists(os.path.join('dumps', ts)):
                    os.makedirs('dumps/'+ts)
                with open(os.path.join('dumps/'+ts+'/valid_E%i.json' % epoch), 'w') as dump_file:
                    json.dump(dump,dump_file)

            # save checkpoint
            if split == 'train':
                checkpoint_path = os.path.join(save_model_path, "E%i.pytorch" % epoch)
                torch.save(model.state_dict(), checkpoint_path)
                print("Model saved at %s" % checkpoint_path)

In [3]:
from cfg import train_cfg

In [4]:
main(train_cfg)

Creating new TRAIN llps data.
Vocablurary of 24 keys created.
Creating new VALID llps data.
SentenceVAE(
  (embedding): Embedding(24, 256)
  (embedding_dropout): Dropout(p=0.3, inplace=False)
  (encoder_rnn): GRU(256, 256, batch_first=True)
  (decoder_rnn): GRU(256, 256, batch_first=True)
  (hidden2mean): Linear(in_features=256, out_features=64, bias=True)
  (hidden2logv): Linear(in_features=256, out_features=64, bias=True)
  (latent2hidden): Linear(in_features=64, out_features=256, bias=True)
  (outputs2vocab): Linear(in_features=256, out_features=24, bias=True)
)


TypeError: only integer tensors of a single element can be converted to an index